In [1]:
# Break in a little-bit
# We want the Java-level operation here, that's all
# !pip install langgraph
from src.utils import *

In [13]:
from playwright.sync_api import sync_playwright

import asyncio
from playwright.async_api import async_playwright

async def run():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()
        url = "https://www.google.com"
        await page.goto(url)

        with open("./src/mark_page.js") as f:
            mark_page_script = f.read()
        await page.evaluate(mark_page_script)
        bboxes = await page.evaluate("markPage()")
        print("Bounding Boxes: ", bboxes)
        screenshot = await page.screenshot()
        print("ScreenShot: ", screenshot)


        await browser.close()

        

# Since this is a Jupyter notebook, we should avoid using asyncio.run() directly in the main block.
# Instead, we can use the following approach to run the async function in notebooks:
import nest_asyncio
nest_asyncio.apply()

if __name__ == "__main__":
    asyncio.get_event_loop().run_until_complete(run())


Bounding Boxes:  [{'x': 44.29296875, 'y': 30, 'type': 'a', 'text': 'About', 'ariaLabel': ''}, {'x': 99.3203125, 'y': 30, 'type': 'a', 'text': 'Store', 'ariaLabel': ''}, {'x': 1020.390625, 'y': 30, 'type': 'a', 'text': 'Gmail', 'ariaLabel': 'Gmail '}, {'x': 1073.68359375, 'y': 30, 'type': 'a', 'text': 'Images', 'ariaLabel': 'Search for Images '}, {'x': 1134, 'y': 30, 'type': 'a', 'text': '', 'ariaLabel': 'Google apps'}, {'x': 1214, 'y': 30, 'type': 'span', 'text': 'Sign in', 'ariaLabel': ''}, {'x': 617.5, 'y': 270, 'type': 'textarea', 'text': '', 'ariaLabel': 'Search'}, {'x': 841, 'y': 269, 'type': 'div', 'text': '', 'ariaLabel': ''}, {'x': 863, 'y': 265.5, 'type': 'path', 'text': '', 'ariaLabel': ''}, {'x': 903.0087280273438, 'y': 270, 'type': 'circle', 'text': '', 'ariaLabel': ''}, {'x': 562.80078125, 'y': 341, 'type': 'input', 'text': '', 'ariaLabel': 'Google Search'}, {'x': 709.640625, 'y': 341, 'type': 'input', 'text': '', 'ariaLabel': "I'm Feeling Lucky"}, {'x': 624.78515625, 'y':